In [10]:
from dataclasses import dataclass,field
import os.path
from os import listdir
import sys
from enum import Enum
import importlib
import xlrd
from xlutils.copy import copy
import json
import jsons
import math
import requests
import datetime
import gzip

from collections import OrderedDict

#Change input_directory to Elite Insight log directory
input_directory = 'D:\\GW2_Logs\\Output\\'
files = listdir(input_directory)
sorted_files = sorted(files)

skill_Dict = {}
downed_Healing = {}
logCount = 0

for filename in sorted_files:
    logCount +=1
    # skip files of incorrect filetype
    file_start, file_extension = os.path.splitext(filename)
    #if args.filetype not in file_extension or "top_stats" in file_start:
    if file_extension not in ['.json', '.gz'] or "top_stats" in file_start:
        continue

    print_string = "parsing "+filename+" - Log Count:"+str(logCount)
    print(print_string)
    file_path = "".join((input_directory,"/",filename))

    if file_extension == '.gz':
        with gzip.open(file_path, mode="r") as f:
            json_data = json.loads(f.read().decode('utf-8'))
    else:
        json_datafile = open(file_path, encoding='utf-8')
        json_data = json.load(json_datafile)    
    
    players = json_data['players']

    #create dictionary of skill_ids and skill_names

    skills = json_data['skillMap']
    for skill_id, skill in skills.items():
        x_id=skill_id[1:]
        if x_id not in skill_Dict:
            skill_Dict[x_id] = skill['name']
    skillBuffs = json_data['buffMap']
    for skill_id, skill in skillBuffs.items():
        x_id=skill_id[1:]
        if x_id not in skill_Dict:
            skill_Dict[x_id] = skill['name']    
    
    for player in players:
        if player['notInSquad']:
            continue
        squadDps_name = player['name']
        squadDps_profession = player['profession']
        squadDps_prof_name = "{{"+squadDps_profession+"}} "+squadDps_name

        if 'extHealingStats' in player:
            i = 0
            for target in player['extHealingStats']['totalHealingDist'][0]:
                targetName = players[i]['name']
                targetGroup = players[i]['group']
                i+1
                if 'totalDownedHealing' in target:
                    if int(target['totalDownedHealing']) > 0:
                        x_id = str(target['id'])
                        try: 
                            reviveSkill = skill_Dict[str(x_id)]
                        except:
                            reviveSkill = "Unknown-"+str(target['id'])

                        if reviveSkill not in downed_Healing:
                            downed_Healing[reviveSkill] = {}
                            downed_Healing[reviveSkill]['Heals'] = {}
                            downed_Healing[reviveSkill]['Hits'] = {}
                            downed_Healing[reviveSkill]['Heals'] = int(target['totalDownedHealing'])
                            downed_Healing[reviveSkill]['Hits'] = int(target['hits'])
                        else:
                            downed_Healing[reviveSkill]['Heals'] = downed_Healing[reviveSkill]['Heals'] + int(target['totalDownedHealing'])
                            downed_Healing[reviveSkill]['Hits'] = downed_Healing[reviveSkill]['Hits'] + int(target['hits'])	
                        #print(targetName, targetGroup, reviveSkill, int(target['totalDownedHealing']))
print('=====Complete=====')
        #End Instant Revive tracking
    
print("|Skill | Downed Healing| Skill Hits|h")
for skill in downed_Healing:
    print("|"+skill+" | "+str(downed_Healing[skill]['Heals'])+"| "+str(downed_Healing[skill]['Hits'])+"|")
    

parsing 20240724-220004_detailed_wvw_kill.json.gz - Log Count:425
=====Complete=====
|Skill | Downed Healing| Skill Hits|h
